# REMIND Electricity Sector Biosphere Flows

Calculate all biosphere flow coefficients for the REMIND electricity sector. **Values are given per kWh**.

In [ ]:
from progressbar import progressbar
import helpers.eimod as eimod
import helpers.ei2rmnd as ei2rmnd

In [ ]:
%run initialize_notebook.ipynb

In [ ]:
databases

Add some inventory material flows to the biosphere:

In [ ]:
from helpers import activitymaps
activitymaps.materials

In [ ]:
ei2rmnd.add_REMIND_technosphere_flows()

## Bioflows of the electricity sector without double counting

Calculate impact for an electricity generating technology in the REMIND power sector excluding *other* activities that are part of the electricity sector to avoid double counting.

In [ ]:
df = ei2rmnd.rmnd_bioflows(activitymaps.powerplants, "EUR", "BAU", 2015)

In [ ]:
df = df.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df.sample(20)

## Bioflows from the electricity sector with double counting

E.g., flows from wind power plants include coal-generated electricity impacts that are already being accounted for by a `electricity production, coal` lca.

In [ ]:
df_dc = rmnd_bioflows("EUR", double_counting=True)

In [ ]:
df_dc = df_dc.reset_index().rename({"level_0": "RMND Tech"}, axis=1).set_index(["RMND Tech", "flow"])
df_dc.sample(20)

## Impact of double counting

Compare versions with and without double counting.

In [ ]:
fulldf = df.join(df_dc, lsuffix="_nodc", rsuffix="_dc")

In [ ]:
fulldf["rel. (nodc/dc)"] = fulldf["amount_nodc"]/fulldf["amount_dc"]

In [ ]:
fulldf["abs. (dc - nodc)"] = fulldf["amount_dc"] - fulldf["amount_nodc"]

In [ ]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 0].sort_values("rel. (nodc/dc)", ascending=False)[:30]

In [ ]:
fulldf.loc[fulldf["rel. (nodc/dc)"] > 1].sort_values(by="rel. (nodc/dc)", ascending=False)

### Only technosphere / inventory flows

In [ ]:
techs = ["steel", "concrete", "copper", "aluminium", "electricity", 
         "gas", "diesel", "petrol", "freight", "cement", "heat"]
techdf = fulldf.reset_index()

In [ ]:
techdf[techdf["flow"].isin(techs)].set_index(["RMND Tech", "flow"])

### What about geothermal power plants?

...seem to have quite a large double-counting value (large electricity production?).

In [ ]:
databases

In [ ]:
eidb = Database("ecoinvent_Remind_BAU_2030")

In [ ]:
regions = ei2rmnd.ei_locations_in_remind_region("EUR")
geot = [act for act in eidb if "electricity production" in act["name"]
and "geothermal" in act["name"]
       and act["location"] in regions]

In [ ]:
geot

In [ ]:
lca = LCA({act: 1./len(geot) for act in geot})

In [ ]:
lca.lci()

In [ ]:
bio = Database("biosphere3")

In [ ]:
ant=[act for act in bio if "Antimony" == act["name"]]

In [ ]:
ant_idcs = []
for act in ant:
    try:
        ant_idcs.append(lca.biosphere_dict[act.key])
    except KeyError as e:
        print("Flow not found: {}".format(get_activity(act.key)["name"]))

In [ ]:
ant_contr = lca.inventory[ant_idcs].sum(axis=0).tolist()[0]

In [ ]:
names = [get_activity(key)["name"] for key in lca.activity_dict]

In [ ]:
contrbtrs = pd.Series({"{} | {}".format(names[idx], list(lca.activity_dict.keys())[idx]): ant_contr[idx] for idx in range(len(names))})

In [ ]:
contrbtrs.sort_values()[:20]

In [ ]:
contrbtrs.sum()

The negative impact seems to come from bioenergy power plants.